# Deep Pensieve™
A Residual Multi-Stage Maximum Mean Discrepancy Variational Resize-Convolution Auto-Encoder with Group Normalization (RMSMMDVRCAECwGN)

## Imports

In [1]:
import time
import json
import random
import numpy as np
import tensorflow as tf

from libs import utils, gif
from libs.group_norm import GroupNormalization

from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Flatten, Reshape, Add, Multiply, Activation, Lambda
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.callbacks import LambdaCallback

from keras import optimizers
from keras import backend as K

from keras_contrib.losses import DSSIMObjective

Using TensorFlow backend.


## Load Images

In [2]:
DIRECTORY = 'roadtrip'

SIZE = 256
CHANNELS = 3
FEATURES = SIZE*SIZE*CHANNELS

MODEL_NAME = DIRECTORY+'-'+str(SIZE)

In [3]:
# load images
imgs, xs, ys = utils.load_images(directory="imgs/"+DIRECTORY,rx=SIZE,ry=SIZE)

# normalize pixels
IMGS = imgs/127.5 - 1
FLAT = np.reshape(IMGS,(-1,FEATURES))
SAMPLES =  np.random.permutation(FLAT)[:9]
TOTAL_BATCH = IMGS.shape[0]

# print shapes
print("MODEL: ",MODEL_NAME)
print("IMGS: ",IMGS.shape)
print("FLAT: ",FLAT.shape)
print("SAMPLES: ",SAMPLES.shape)

Loading images:	184
MODEL:  roadtrip-256
IMGS:  (184, 256, 256, 3)
FLAT:  (184, 196608)
SAMPLES:  (9, 196608)


## Maximum Mean Discrepancy 

https://ermongroup.github.io/blog/a-tutorial-on-mmd-variational-autoencoders/

Maximum mean discrepancy (MMD, (Gretton et al. 2007)) is based on the idea that two distributions are identical if and only if all their moments are the same. Therefore, we can define a divergence by measuring how “different” the moments of two distributions p(z) and q(z) are. MMD can accomplish this efficiently via the kernel embedding trick:

A kernel can be intuitively interpreted as a function that measures the “similarity” of two samples. It has a large value when two samples are similar, and small when they are different. For example, the Gaussian kernel considers points that are close in Euclidean space to be “similar”. A rough intuition of MMD, then, is that if two distributions are identical, then the average “similarity” between samples from each distribution, should be identical to the average “similarity” between mixed samples from both distributions.

In [4]:
def compute_kernel(x, y):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float32))

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

## Encoder

In [5]:
def encode(x):
    # set current layer
    current_layer = Reshape((SIZE,SIZE,CHANNELS))(x)
    
    # convolution layers
    for layer, n_filters in enumerate(FILTERS):

        # stacked 3x3 convolutions with group normalization + activation
        current_layer = Conv2D(n_filters,3,padding='SAME',kernel_initializer=INITIALIZER)(current_layer)
        current_layer = GroupNormalization(groups=n_filters,axis=-1)(current_layer)
        current_layer = Activation(ACTIVATION)(current_layer)

        current_layer = Conv2D(n_filters,3,padding='SAME',kernel_initializer=INITIALIZER)(current_layer)
        current_layer = GroupNormalization(groups=n_filters,axis=-1)(current_layer)
        current_layer = Activation(ACTIVATION)(current_layer)
         
        # max pooling
        current_layer = MaxPooling2D()(current_layer)
    
    # grab the last shape for reconstruction
    shape = current_layer.get_shape().as_list()
    
    # flatten
    flat = Flatten()(current_layer)
    
    # latent vector
    z = Dense(LATENT_DIM,name='encoder')(flat)
    
    return z, (shape[1],shape[2],shape[3])

## Decoder

In [6]:
def decode(z,z_g,shape=None):
    
    # reverse the encoder
    filters = FILTERS[::-1]

    # inflate
    inflated = shape[0]*shape[1]*shape[2]
    inflate = Dense(inflated,name='generator')
    current_layer = inflate(z) ; generator = inflate(z_g)
    
    # reshape
    reshape = Reshape(shape)
    current_layer = reshape(current_layer) ; generator = reshape(generator)
    
    # build layers
    for layer, n_filters in enumerate(filters):
        
        # upsample
        u = UpSampling2D()
        current_layer = u(current_layer) ; generator = u(generator)

        # stacked 3x3 convolutions with group normalization + activation
        c1 = Conv2D(n_filters,3,padding='SAME',kernel_initializer=INITIALIZER)
        b1 = GroupNormalization(groups=n_filters,axis=-1)
        a1 = Activation(ACTIVATION)

        current_layer = c1(current_layer) ; generator = c1(generator)
        current_layer = b1(current_layer) ; generator = b1(generator)
        current_layer = a1(current_layer) ; generator = a1(generator)

        c2 = Conv2D(n_filters,3,padding='SAME',kernel_initializer=INITIALIZER)
        b2 = GroupNormalization(groups=n_filters,axis=-1)
        a2 = Activation(ACTIVATION)

        current_layer = c2(current_layer) ; generator = c2(generator)
        current_layer = b2(current_layer) ; generator = b2(generator)
        current_layer = a2(current_layer) ; generator = a2(generator)
    
    # output convolution + activation
    conv = Conv2D(CHANNELS,1,padding='SAME')
    activation = Activation('tanh',name='decoder_dssim')
    
    current_layer = conv(current_layer)       ; generator = conv(generator)
    current_layer = activation(current_layer) ; generator = activation(generator)
    
    flatten = Flatten(name='decoder')
    decoder_loss = flatten(current_layer)
    
    return current_layer, generator, decoder_loss

## Residual

In [7]:
def residual(x,x_g):
    
    current_layer = x ; generator = x_g

    # shortcuts
    shortcut = current_layer 
    shortcut_g = generator

    # conv 1
    c1 = Conv2D(R_FILTERS,3,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c1(current_layer) ; generator = c1(generator)
    
    # activation 1
    a1 = Activation(ACTIVATION)
    current_layer = a1(current_layer) ; generator = a1(generator)

    # conv 2
    c2 = Conv2D(R_FILTERS,3,padding='SAME',kernel_initializer=INITIALIZER)
    current_layer = c2(current_layer) ; generator = c2(generator)

    # residual scaling
    scale = Lambda(lambda x: x * R_SCALING)
    current_layer = scale(current_layer) ; generator = scale(generator)

    # fix shortcut shape if mismatch
    if(shortcut.shape[-1] != current_layer.shape[-1]):
        s = Conv2D(R_FILTERS,1,padding='SAME',kernel_initializer=INITIALIZER)
        shortcut = s(current_layer) ; shortcut_g = s(generator)
        
    # merge shortcut
    merge = Add()
    current_layer = merge([current_layer, shortcut]) ; generator = merge([generator, shortcut_g])

    return current_layer, generator

## Refiner

In [8]:
def refine(x,x_g):
    
    # reshape
    reshape = Reshape((SIZE,SIZE,CHANNELS))
    current_layer = reshape(x) ; generator = reshape(x_g)

    # residual layers
    for layer in range(R_LAYERS):
    
        # residual block
        current_layer, generator = residual(current_layer,generator)
    
    # output convolution + activation
    conv = Conv2D(CHANNELS,1,padding='SAME')
    activation = Activation('tanh',name='refiner_dssim')
    
    current_layer = conv(current_layer)       ; generator = conv(generator)
    current_layer = activation(current_layer) ; generator = activation(generator)
    
    flatten = Flatten(name='refiner')
    refiner_loss = flatten(current_layer)
    
    return current_layer, generator, refiner_loss

## Callbacks

In [9]:
RECONS = []

def gifit(epoch=None):
    if (epoch % GIF_STEPS == 0):
        print('saving gif ...')
        z,y,yc,i,ic = AUTOENCODER.predict_on_batch(SAMPLES)
        img = np.clip(127.5*(i+1).reshape((-1, SIZE, SIZE, CHANNELS)), 0, 255)
        RECONS.append(utils.montage(img).astype(np.uint8))
        
def saveit(epoch=None):
    if (epoch == 0):
        print('saving model ...')
        with open(MODEL_NAME+'-autoencoder-model.json', "w") as json_file:
            json_file.write(AUTOENCODER.to_json())
        AUTOENCODER.save(MODEL_NAME+'-autoencoder-model.h5')
        
        print('saving encoder ...')          
        with open(MODEL_NAME+'-encoder-model.json', "w") as json_file:
            json_file.write(ENCODER.to_json())
        ENCODER.save(MODEL_NAME+'-encoder-model.h5')
        
        print('saving generator ...')
        with open(MODEL_NAME+'-generator-model.json', "w") as json_file:
            json_file.write(GENERATOR.to_json())
        GENERATOR.save(MODEL_NAME+'-generator-model.h5')
    
    elif (epoch % MODEL_STEPS == 0):
        print('saving autoencoder weights ...')
        AUTOENCODER.save_weights(MODEL_NAME+'-autoencoder-weights.h5')
        AUTOENCODER.save(MODEL_NAME+'-autoencoder-model.h5')
        
        print('saving encoder weights ...')
        ENCODER.save_weights(MODEL_NAME+'-encoder-weights.h5')
        ENCODER.save(MODEL_NAME+'-encoder-model.h5')
        
        print('saving generator weights ...')
        GENERATOR.save_weights(MODEL_NAME+'-generator-weights.h5')
        GENERATOR.save(MODEL_NAME+'-generator-model.h5')

## Model

In [10]:
# 512px
# FILTERS = [64,80,96,112,96,80,64]

# 256px
FILTERS = [64,96,128,160,128,64]

#128px
#FILTERS = [64,128,196,128,64]

# 64px
#FILTERS = [32,48,64,48]

# Residuals
R_LAYERS  = 24
R_FILTERS = 64
R_SCALING = 0.1

# Default initializer and activation
INITIALIZER = 'he_normal'
ACTIVATION  = 'elu'

# Latent dimension size
LATENT_DIM = 1024

## Loss Functions

In [11]:
def vae_loss(y_true,y_pred):
    epsilon = tf.random_normal(tf.stack([BATCH_SIZE, LATENT_DIM]))
    latent_loss = compute_mmd(epsilon, y_pred)
    return latent_loss

## Training

In [15]:
EPOCHS      = 12501
BATCH_SIZE  = 4

MODEL_STEPS = 10
GIF_STEPS   = 100

SAMPLES =  np.random.permutation(FLAT)[:9]

In [ ]:
# input
X = Input(shape=(FEATURES,),name='input')

# latent
Z, shape = encode(X)

# generator input
Z_G = Input(shape=(LATENT_DIM,))

# coarse reconstruction
Y, YG, Y_F = decode(Z,Z_G,shape)

# fine reconstruction
IMG, IMG_G, IMG_F = refine(Y,YG)
    
# define autoencoder
AUTOENCODER = Model(inputs=[X], outputs=[Z,Y,Y_F,IMG,IMG_F])

# define encoder
ENCODER = Model(inputs=[X], outputs=[Z])

# define generator
GENERATOR = Model(inputs=[Z_G], outputs=[IMG_G])

# define optimizer
ADAM = optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=True)

# define losses
losses = {'encoder':vae_loss,
          'decoder':'mse','decoder_dssim':DSSIMObjective(),
          'refiner':'mae','refiner_dssim':DSSIMObjective() }

# compile models
AUTOENCODER.compile(optimizer=ADAM,loss=losses)
ENCODER.compile(optimizer=ADAM,loss='mse')
GENERATOR.compile(optimizer=ADAM,loss='mse')

# print summary
AUTOENCODER.summary()

# callbacks
giffer = LambdaCallback(on_epoch_end=lambda epoch, logs: gifit(epoch))
saver = LambdaCallback(on_epoch_end=lambda epoch, logs: saveit(epoch))

# fit model
AUTOENCODER.fit(x=FLAT,y=[FLAT,IMGS,FLAT,IMGS,FLAT],batch_size=BATCH_SIZE,epochs=EPOCHS,callbacks=[giffer,saver])

# save animated gif
gif.build_gif(RECONS, saveto=MODEL_NAME+'-final'+ "-"+str(time.time())+'.gif')

print("done")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 196608)       0                                            
__________________________________________________________________________________________________
reshape_7 (Reshape)             (None, 256, 256, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
conv2d_151 (Conv2D)             (None, 256, 256, 64) 1792        reshape_7[0][0]                  
__________________________________________________________________________________________________
group_normalization_49 (GroupNo (None, 256, 256, 64) 128         conv2d_151[0][0]                 
__________________________________________________________________________________________________
activation

Epoch 1/12501
184/184 [==============================] - 42s 227ms/step - loss: 1.7938 - encoder_loss: 0.0049 - decoder_dssim_loss: 0.4848 - decoder_loss: 0.3711 - refiner_dssim_loss: 0.4888 - refiner_loss: 0.4443
saving gif ...
saving model ...
saving encoder ...
saving generator ...
Epoch 2/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.5606 - encoder_loss: 0.0055 - decoder_dssim_loss: 0.4684 - decoder_loss: 0.2402 - refiner_dssim_loss: 0.4649 - refiner_loss: 0.3816
Epoch 3/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.4441 - encoder_loss: 0.0075 - decoder_dssim_loss: 0.4648 - decoder_loss: 0.1820 - refiner_dssim_loss: 0.4539 - refiner_loss: 0.3358
Epoch 4/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.4438 - encoder_loss: 0.0054 - decoder_dssim_loss: 0.4720 - decoder_loss: 0.1883 - refiner_dssim_loss: 0.4561 - refiner_loss: 0.3220
Epoch 5/12501
184/184 [==============================] - 27s 148ms/step 

184/184 [==============================] - 27s 148ms/step - loss: 1.1439 - encoder_loss: 0.0045 - decoder_dssim_loss: 0.3807 - decoder_loss: 0.1128 - refiner_dssim_loss: 0.3853 - refiner_loss: 0.2606
Epoch 38/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.1364 - encoder_loss: 0.0048 - decoder_dssim_loss: 0.3806 - decoder_loss: 0.1098 - refiner_dssim_loss: 0.3837 - refiner_loss: 0.2575
Epoch 39/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.1063 - encoder_loss: 0.0045 - decoder_dssim_loss: 0.3716 - decoder_loss: 0.1054 - refiner_dssim_loss: 0.3718 - refiner_loss: 0.2531
Epoch 40/12501
184/184 [==============================] - 27s 148ms/step - loss: 1.1160 - encoder_loss: 0.0044 - decoder_dssim_loss: 0.3722 - decoder_loss: 0.1085 - refiner_dssim_loss: 0.3736 - refiner_loss: 0.2573
Epoch 41/12501
184/184 [==============================] - 28s 150ms/step - loss: 1.1029 - encoder_loss: 0.0042 - decoder_dssim_loss: 0.3724 - decoder_loss:

184/184 [==============================] - 27s 148ms/step - loss: 0.9021 - encoder_loss: 0.0031 - decoder_dssim_loss: 0.3057 - decoder_loss: 0.0782 - refiner_dssim_loss: 0.3072 - refiner_loss: 0.2078
Epoch 74/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.9059 - encoder_loss: 0.0029 - decoder_dssim_loss: 0.3083 - decoder_loss: 0.0796 - refiner_dssim_loss: 0.3065 - refiner_loss: 0.2086
Epoch 75/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.9404 - encoder_loss: 0.0029 - decoder_dssim_loss: 0.3211 - decoder_loss: 0.0828 - refiner_dssim_loss: 0.3207 - refiner_loss: 0.2130
Epoch 76/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.9186 - encoder_loss: 0.0032 - decoder_dssim_loss: 0.3126 - decoder_loss: 0.0820 - refiner_dssim_loss: 0.3101 - refiner_loss: 0.2107
Epoch 77/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.9150 - encoder_loss: 0.0030 - decoder_dssim_loss: 0.3117 - decoder_loss:

184/184 [==============================] - 27s 148ms/step - loss: 0.8457 - encoder_loss: 0.0026 - decoder_dssim_loss: 0.2884 - decoder_loss: 0.0709 - refiner_dssim_loss: 0.2896 - refiner_loss: 0.1942
Epoch 110/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.8220 - encoder_loss: 0.0025 - decoder_dssim_loss: 0.2820 - decoder_loss: 0.0688 - refiner_dssim_loss: 0.2803 - refiner_loss: 0.1883
Epoch 111/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.7958 - encoder_loss: 0.0027 - decoder_dssim_loss: 0.2721 - decoder_loss: 0.0659 - refiner_dssim_loss: 0.2713 - refiner_loss: 0.1838
saving autoencoder weights ...
saving encoder weights ...
saving generator weights ...
Epoch 112/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.7999 - encoder_loss: 0.0026 - decoder_dssim_loss: 0.2730 - decoder_loss: 0.0662 - refiner_dssim_loss: 0.2730 - refiner_loss: 0.1851
Epoch 113/12501
184/184 [==============================] - 27s 14

...

184/184 [==============================] - 27s 149ms/step - loss: 0.4005 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1486 - decoder_loss: 0.0223 - refiner_dssim_loss: 0.1425 - refiner_loss: 0.0859
Epoch 1298/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3952 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1471 - decoder_loss: 0.0222 - refiner_dssim_loss: 0.1399 - refiner_loss: 0.0849
Epoch 1299/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.3924 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1465 - decoder_loss: 0.0220 - refiner_dssim_loss: 0.1386 - refiner_loss: 0.0842
Epoch 1300/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3928 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1465 - decoder_loss: 0.0221 - refiner_dssim_loss: 0.1387 - refiner_loss: 0.0844
Epoch 1301/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3954 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1475 - decod

184/184 [==============================] - 28s 149ms/step - loss: 0.3940 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1470 - decoder_loss: 0.0219 - refiner_dssim_loss: 0.1394 - refiner_loss: 0.0845
Epoch 1334/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3901 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1459 - decoder_loss: 0.0219 - refiner_dssim_loss: 0.1376 - refiner_loss: 0.0838
Epoch 1335/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.3909 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1460 - decoder_loss: 0.0219 - refiner_dssim_loss: 0.1381 - refiner_loss: 0.0838
Epoch 1336/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3928 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1466 - decoder_loss: 0.0219 - refiner_dssim_loss: 0.1387 - refiner_loss: 0.0846
Epoch 1337/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3908 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1460 - decod

184/184 [==============================] - 27s 149ms/step - loss: 0.3906 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1456 - decoder_loss: 0.0216 - refiner_dssim_loss: 0.1383 - refiner_loss: 0.0841
Epoch 1370/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3894 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1453 - decoder_loss: 0.0216 - refiner_dssim_loss: 0.1377 - refiner_loss: 0.0837
Epoch 1371/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3869 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1448 - decoder_loss: 0.0216 - refiner_dssim_loss: 0.1365 - refiner_loss: 0.0830
saving autoencoder weights ...
saving encoder weights ...
saving generator weights ...
Epoch 1372/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3871 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1449 - decoder_loss: 0.0215 - refiner_dssim_loss: 0.1365 - refiner_loss: 0.0831
Epoch 1373/12501
184/184 [==============================] - 27

184/184 [==============================] - 27s 148ms/step - loss: 0.3900 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1450 - decoder_loss: 0.0215 - refiner_dssim_loss: 0.1382 - refiner_loss: 0.0842
Epoch 1406/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3877 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1449 - decoder_loss: 0.0214 - refiner_dssim_loss: 0.1368 - refiner_loss: 0.0834
Epoch 1407/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3860 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1445 - decoder_loss: 0.0214 - refiner_dssim_loss: 0.1365 - refiner_loss: 0.0826
Epoch 1408/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.3843 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1442 - decoder_loss: 0.0213 - refiner_dssim_loss: 0.1356 - refiner_loss: 0.0821
Epoch 1409/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3822 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1435 - decod

184/184 [==============================] - 27s 148ms/step - loss: 0.3808 - encoder_loss: 0.0010 - decoder_dssim_loss: 0.1431 - decoder_loss: 0.0211 - refiner_dssim_loss: 0.1338 - refiner_loss: 0.0817
saving autoencoder weights ...
saving encoder weights ...
saving generator weights ...
Epoch 1442/12501
184/184 [==============================] - 27s 149ms/step - loss: 0.3791 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1427 - decoder_loss: 0.0210 - refiner_dssim_loss: 0.1331 - refiner_loss: 0.0813
Epoch 1443/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3811 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1429 - decoder_loss: 0.0210 - refiner_dssim_loss: 0.1341 - refiner_loss: 0.0821
Epoch 1444/12501
184/184 [==============================] - 27s 148ms/step - loss: 0.3837 - encoder_loss: 0.0011 - decoder_dssim_loss: 0.1435 - decoder_loss: 0.0211 - refiner_dssim_loss: 0.1351 - refiner_loss: 0.0829
Epoch 1445/12501
184/184 [==============================] - 28

## Load Models

In [ ]:
print('loading encoder ...')
#ENCODER = load_model(MODEL_NAME+'-encoder.hdf5')

print('loading generator ...')
#GENERATOR = load_model(MODEL_NAME+'-generator.hdf5')

print('done')

## Reconstruction 

In [ ]:
# # ENCODER = load_model(MODEL_NAME+'-encoder.hdf5')
from keras.models import model_from_json

# print('saving encoder ...')
# ENCODER.save_weights(MODEL_NAME+'-encoder-weights.h5')
# GENERATOR.save_weights(MODEL_NAME+'-generator-weights.h5')
# ENCODER.save(MODEL_NAME+'-encoder-model.h5')
# # GENERATOR.save(MODEL_NAME+'-generator.h5')

# print('loading encoder ...')
# enc_json_file = open(MODEL_NAME+'-encoder-model.json', 'r')
# enc_model_json = enc_json_file.read()
# enc_json_file.close()
# print('loading weights ...')
# enc_loaded_model = model_from_json(enc_model_json)
# enc_loaded_model.load_weights(MODEL_NAME+'-encoder-weights.h5')


# print('loading generator ...')
# gen_json_file = open(MODEL_NAME+'-generator-model.json', 'r')
# gen_model_json = gen_json_file.read()
# gen_json_file.close()
# print('loading weights ...')
# gen_loaded_model = model_from_json(gen_model_json)
# gen_loaded_model.load_weights(MODEL_NAME+'-generator-weights.h5')


# # ENC = load_model(MODEL_NAME+'-encoder-model.h5')
# # GEN = load_model(MODEL_NAME+'-generator.h5')

# # ENC.load_weights(MODEL_NAME+'-encoder.h5')
# # GEN.load_weights(MODEL_NAME+'-generator.h5')

# ENC = enc_loaded_model
# GEN = gen_loaded_model

ENC = ENCODER
GEN = GENERATOR

def reconstruct(index=0):
    x = np.reshape(FLAT[index],(-1,FEATURES))

    z_e = ENC.predict_on_batch(x)
    print(z_e)
    
#     img_g = np.reshape(GEN.predict_on_batch(z),(-1,FEATURES))
    print(GEN.predict_on_batch(z_e).shape)
    img_g_z = np.reshape(GEN.predict_on_batch(z_e),(-1,FEATURES))
    
    t = IMGS[index]/2 + .5
#     img = np.reshape(img/2 + .5,(SIZE,SIZE,CHANNELS))
#     img_g = np.reshape(img_g/2 + .5,(SIZE,SIZE,CHANNELS))
    img_g_z = np.reshape(img_g_z/2 + .5,(SIZE,SIZE,CHANNELS))
    
    print("PSNR: %.3f" % (utils.psnr(t,img_g_z)))
    print("MS-SSIM: %.3f" % (utils.MultiScaleSSIM(np.reshape(t,(1,SIZE,SIZE,CHANNELS)),
                                                 np.reshape(img_g_z,(1,SIZE,SIZE,CHANNELS)),
                                                 max_val=1.)))
#                                     utils.MultiScaleSSIM(np.reshape(t,(1,SIZE,SIZE,CHANNELS)),
#                                                  np.reshape(img,(1,SIZE,SIZE,CHANNELS)),
#                                                  max_val=1.),
#                                    utils.MultiScaleSSIM(np.reshape(t,(1,SIZE,SIZE,CHANNELS)),
#                                                  np.reshape(img_g,(1,SIZE,SIZE,CHANNELS)),
#                                                  max_val=1.),
                                   
    
   
    return t, img_g_z

In [ ]:
r = random.randint(0,TOTAL_BATCH) ; print(r)
orig, img_g_z = reconstruct(r)
utils.showImagesHorizontally(images=[orig,img_g_z])

In [ ]:
K.batch_get_value(AUTOENCODER.weights[0])

In [ ]:
K.batch_get_value(ENC.weights[0])

## Latent  Animation

In [ ]:

def random_latents(n_imgs=3,path='linear',steps=30,slices=1,directory='roadtrip'):
    imgs = np.random.permutation(FLAT)[:n_imgs]
    latent_animation(imgs,steps,slices,path=path)

def latent_animation(imgs=None,steps=None,slices=None,path=None,filename="latent-animation-"):
    # get encodings
    print('getting latent vectors ...')
    latents = []
    for index,img in enumerate(imgs):
        img = np.reshape(img,(-1,FEATURES))
        latent = ENC.predict_on_batch(img)
        latents.append(latent)

    # calculate latent transitions
    print('calculating latent manifold path ...')
    recons = []
    current_step = None
    for i in range(len(latents)-1):
        print("IMG: " + str(i))
        l1 = latents[i]
        l2 = latents[i+1]

        # latent image distance
        image_distance = l2 - l1

        # sine wave for animation steps
        integral = steps*(1+np.cos(np.pi/steps))/np.pi
        normalizer = image_distance/integral

        # start image
        current_step = l1
    
        recons.append(l1)
        for i in range(steps):
            current_step = l1 + i*image_distance/steps
            recons.append(current_step)
            
            if(i > 1 and (i+1) % int(steps/slices) == 0):
                print('reconstructing ... ',i)
                recons = np.reshape(recons,(-1,LATENT_DIM))
                i = GEN.predict_on_batch(recons)

                # de-normalize and clip the output
                final = np.clip((127.5*(i+1)).reshape((-1,SIZE,SIZE,CHANNELS)),0,255)

                # build the gif
                filename = filename+str(time.time())
                gif.build_gif([utils.montage([r]).astype(np.uint8) for r in final], saveto=filename+"-final.gif",dpi=128)

                print(filename)
                
                recons = []
                filename="latent-animation-"
        recons.append(l2)

In [ ]:
random_latents(n_imgs=5,steps=100,slices=1)

In [ ]:
imgs =  np.random.permutation(FLAT)
for i in range(TOTAL_BATCH):
    print(i)
    latent_animation([imgs[i],imgs[i+1]],40)
    

## Load Model & Continue Training 

In [ ]:
import keras

giffer = LambdaCallback(on_epoch_end=lambda epoch, logs: gifit(epoch))
saver = LambdaCallback(on_epoch_end=lambda epoch, logs: saveit(epoch))

print('loading model ...')
AUTOENCODER = load_model(MODEL_NAME+'-autoencoder-model.h5',
                         custom_objects={'vae_loss': vae_loss, 
                                         'R_SCALING':0.1, 
                                         'DSSIMObjective':DSSIMObjective()})

# define encoder
ENCODER = Model(inputs=[AUTOENCODER.input], outputs=[AUTOENCODER.get_layer("encoder").output])

# define generator
Z = Input(shape=(LATENT_DIM,))
GENERATOR = Model(inputs=[Z], outputs=[AUTOENCODER.get_layer("generator")(Z)])

ENCODER.compile(optimizer=ADAM,loss='mse')
GENERATOR.compile(optimizer=ADAM,loss='mse')

print('resuming training ...')
AUTOENCODER.fit(x=FLAT,y=[FLAT,IMGS,FLAT,IMGS,FLAT],batch_size=BATCH_SIZE,epochs=EPOCHS,callbacks=[giffer,saver])